In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

In [2]:
print(pyspark.__version__)

3.2.1


In [3]:
url = 'mongodb://mongodb:27017/airlines.flys'

In [4]:
spark = (SparkSession
         .builder
         .master('local[*]')
         .config('spark.driver.extraClassPath','path_to_jars/*')
         .config("spark.mongodb.read.connection.uri",url)
         .config("spark.mongodb.write.connection.uri", url)
         .getOrCreate()
         )

In [5]:
df = spark.read.format("mongodb").load()

In [6]:
df.show()

+--------------------+------+---------------+--------------+----+----+--------------------+
|                 _id|active|       callsign|       country|iata|icao|                name|
+--------------------+------+---------------+--------------+----+----+--------------------+
|649c5998abcec416b...|     Y|               |              |   -| N/A|      Private flight|
|649c5998abcec416b...|     N|        GENERAL| United States|    | GNL|         135 Airways|
|649c5998abcec416b...|     Y|        NEXTIME|  South Africa|  1T| RNX|       1Time Airline|
|649c5998abcec416b...|     N|               |United Kingdom|    | WYT|2 Sqn No 1 Elemen...|
|649c5998abcec416b...|     N|               |        Russia|    | TFU|     213 Flight Unit|
|649c5998abcec416b...|     N| CHKALOVSK-AVIA|        Russia|    | CHD|223 Flight Unit S...|
|649c5998abcec416b...|     N|     CARGO UNIT|        Russia|    | TTF|   224th Flight Unit|
|649c5998abcec416b...|     N|   CLOUD RUNNER|United Kingdom|    | TWF|         2

In [7]:
df.createOrReplaceTempView("airlines")

In [8]:
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- active: string (nullable = true)
 |-- callsign: string (nullable = true)
 |-- country: string (nullable = true)
 |-- iata: string (nullable = true)
 |-- icao: string (nullable = true)
 |-- name: string (nullable = true)



In [9]:
df_group = spark.sql("""SELECT country, count(*) as Numero
            FROM airlines
            group by country """)

In [10]:
df_group.show()

+--------------------+------+
|             country|Numero|
+--------------------+------+
|                    |    15|
|       United States|  1080|
|        South Africa|    91|
|      United Kingdom|   407|
|              Russia|   230|
|            Thailand|    48|
|              Canada|   318|
|           Australia|    93|
|           Singapore|    11|
|             Belgium|    27|
|              Mexico|   439|
|               Spain|   166|
|              France|   119|
|United Arab Emirates|    31|
|   Republic of Korea|     8|
|            Pakistan|    29|
|               Libya|    26|
|              Gambia|     6|
|         Ivory Coast|    17|
|             Ukraine|    89|
+--------------------+------+
only showing top 20 rows



In [11]:
df_group.write.parquet("Resultados.parquet") 